In [1]:
import os, sys, math, datetime
import pathlib
from pathlib import Path
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt
import PIL
import PIL.Image
import seaborn as sns

import tensorflow as tf
# import tensorflow_datasets as tfds
from tensorflow import keras
# from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D,DepthwiseConv2D, MaxPooling2D, AvgPool2D, GlobalAveragePooling2D, BatchNormalization, Concatenate
# from tensorflow.keras.layers import ReLU
# from tensorflow.keras.models import Model
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
 
# Import the necessary MLTK APIs
from mltk.core import view_model, summarize_model, profile_model

# import workbench.config.config
from workbench.config.config import initialize
from workbench.utils.utils import create_filepaths, get_file_size
from workbench.utils.utils import parse_model_name, parse_mltk_model_summary
from workbench.data.data import get_lemon_quality_dataset
from workbench.tflite_profiling import get_peak_memory_df, get_tensor_details_df

#from dotenv import load_dotenv
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint
#import deeplake

%load_ext autoreload



In [2]:
%reload_ext autoreload
%autoreload

In [3]:
models_dir = initialize()

In [4]:
global model_name
#model_name = "efficientNetB0_1_96_c3_o3_keras"
#model_name = "shufflenetv2tinys_0.2_96_c3_o3_f4l1024"
#model_name = "mobilenetv2_1_224_c3_o3_l5"
model_name = "mobilenetv2_1_224_c3_o3_keras"
#model_name = "shufflenetv2tiny_0.1_96_c3_o3_f4l1024"
#model_name = "MobilenetV3small_1_96_c3_o3_keras"#, "MobilenetV3large_1_224_c3_o3_keras"# ,


In [5]:
base_model_name, alpha, resolution, channels, classes, variation = model_name.split("_")
alpha = float(alpha)
resolution = int(resolution)
classes = int(classes.strip("o"))
channels = int(channels.strip("c"))
PROJECT = "model_DB"

# just needed for INT 8 quantization:
BATCH_SIZE = 32
shuffle_seed = 1

In [6]:
models_path, models_summary_path, models_image_path, models_layer_df_path, models_tf_path, models_tflite_path, models_tflite_opt_path = create_filepaths(model_name)

i:\tinyml\tiny_cnn\models


In [7]:
model = keras.models.load_model(models_tf_path)

In [8]:
# view_model(model, tflite=True, build=True)

# Parse model parameters from MLTK summary

In [9]:
total_params, trainable_params, non_trainable_params, MACs, FLOPs = parse_mltk_model_summary(models_summary_path)        

In [10]:
mltk_model_stats = {}

In [11]:
mltk_model_stats["MACs"] = MACs
mltk_model_stats["FLOPs"] = FLOPs
mltk_model_stats["total_params"] = total_params
mltk_model_stats["trainable_params"] = trainable_params
mltk_model_stats["non_trainable_params"] = non_trainable_params

# Creating tflite Models

In [12]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
#converter = tf.lite.TFLiteConverter.from_saved_model(models_path)
tflite_model = converter.convert()


INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmpm53ud36z\assets


INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmpm53ud36z\assets


In [13]:
# Save the model.
with open(models_tflite_path, "wb") as f:
    f.write(tflite_model)

In [14]:
mltk_model_stats["model_size_kb"] = get_file_size(models_tf_path)
mltk_model_stats["tflite_model_size_kb"] = get_file_size(models_tflite_path)

File size in bytes is 9523096
File size in kilobytes is 9299.8984375
File size in bytes is 8826380
File size in kilobytes is 8619.51171875


# tflite with INT8 quantization

In [15]:
dataset_path = Path.cwd().joinpath("datasets", "lemon_dataset")
dataset_path.exists()

True

In [16]:
train_ds, val_ds, test_ds, labels = get_lemon_quality_dataset(dataset_path, resolution, resolution, BATCH_SIZE, channels)

Color mode: rgb
Preparing training dataset...
Found 2021 files belonging to 3 classes.
Preparing validation dataset...
Found 252 files belonging to 3 classes.
Preparing test dataset...
Found 255 files belonging to 3 classes.
Class names: ['bad_quality', 'empty_background', 'good_quality']
Train: (TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))
Normalize: True


In [17]:
repr_ds = test_ds.unbatch()

def representative_data_gen():
  for i_value, o_value in repr_ds.batch(1).take(48):
    yield [i_value]
    
converter_opt = tf.lite.TFLiteConverter.from_keras_model(model)

# set the optimization flag
converter_opt.optimizations = [tf.lite.Optimize.DEFAULT]
# enforce integer only quantization
converter_opt.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter_opt.inference_input_type = tf.uint8
converter_opt.inference_output_type = tf.uint8

# provide a representative dataset for quantization
converter_opt.representative_dataset = representative_data_gen

tflite_model_opt = converter_opt.convert()

# Save the model.
with open(models_tflite_opt_path, 'wb') as f:
  f.write(tflite_model_opt)
models_tflite_opt_path

INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmp4u5xansv\assets


INFO:tensorflow:Assets written to: C:\Users\Susanne\AppData\Local\Temp\tmp4u5xansv\assets
d:\Miniconda\envs\tiny_cnn_6\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


WindowsPath('i:/tinyml/tiny_cnn/models/mobilenetv2_1_224_c3_o3_keras/mobilenetv2_1_224_c3_o3_keras_INT8.tflite')

In [18]:
mltk_model_stats["tflite_model_INT8_size_kb"] = get_file_size(models_tflite_opt_path)

File size in bytes is 2665256
File size in kilobytes is 2602.7890625


In [19]:
mltk_model_stats["tflite_model_size_kb"] = get_file_size(models_tflite_path)

File size in bytes is 8826380
File size in kilobytes is 8619.51171875


# Profiling tflite models with MLTK profiler

In [20]:
mltk_model_stats["arhcitecture"] = model_name
mltk_model_stats["alpha"] = alpha
mltk_model_stats["img_res"] = resolution
mltk_model_stats["classes"] = classes
mltk_model_stats["channels"] = channels
mltk_model_stats["variation"] = variation

In [21]:
# MLTK profile model reads the mode from a path!

profiling_results = profile_model(str(models_tflite_path), accelerator=None, build=None)

Profiling model in simulator ...
Using Tensorflow-Lite Micro version: b13b48c (2022-06-08)
Searching for optimal runtime memory size ...
Determined optimal runtime memory size to be 9871728


In [22]:
mltk_model_stats["opt_RAM_mltk"] = int(input("Please copy the optimal runtime memory size from above: \n"))

In [23]:
profiling_results.get_summary(exclude_null=False, full_summary=True)
model_profile = profiling_results.to_dict()

In [24]:
mltk_model_stats["flash_model_size_b_mltk"] = model_profile["summary"]["tflite_size"]
mltk_model_stats["RAM_runtime_memory_size_b_mltk"] = model_profile["summary"]["runtime_memory_size"]
mltk_model_stats["OPS_mltk"] = model_profile["summary"]["ops"]
mltk_model_stats["macs_mltk"] = model_profile["summary"]["macs"]
mltk_model_stats["n_unsupported_layers_mltk"] = model_profile["summary"]["n_unsupported_layers"]
mltk_model_stats["energy_mltk"] = model_profile["summary"]["energy"]

In [25]:
layers_df = pd.DataFrame.from_dict(model_profile["layers"])
layers_df["name"] = layers_df["index"].map(str) + ": " + layers_df["opcode"]
layers_df.insert(2,"name" , layers_df.pop("name"))
layers_df

,index,opcode,name,ops,macs,cpu_cycles,energy,input_shape,output_shape,options
0,0,conv_2d,0: conv_2d,22880256,10838016,1.352801e+08,2.802536e-02,"1x224x224x3,32x3x3x3,32",1x112x112x32,Padding:same stride:2x2 activation:relu6
1,1,depthwise_conv_2d,1: depthwise_conv_2d,8429568,3612672,1.429125e+07,2.795809e-03,"1x112x112x32,1x3x3x32,32",1x112x112x32,Multiplier:1 padding:same stride:1x1 activatio...
2,2,conv_2d,2: conv_2d,13045760,6422528,1.957062e+07,3.881317e-03,"1x112x112x32,16x1x1x32,16",1x112x112x16,Padding:same stride:1x1 activation:none
3,3,conv_2d,3: conv_2d,42147840,19267584,4.246512e+07,8.768505e-03,"1x112x112x16,96x1x1x16,96",1x112x112x96,Padding:same stride:1x1 activation:relu6
4,4,pad,4: pad,7354944,0,2.119680e+06,4.140765e-04,"1x112x112x96,4x2",1x113x113x96,Type=padoptions
...,...,...,...,...,...,...,...,...,...,...
64,64,conv_2d,64: conv_2d,30121280,15052800,9.250128e+06,1.780007e-03,"1x7x7x960,320x1x1x960,320",1x7x7x320,Padding:same stride:1x1 activation:none
65,65,conv_2d,65: conv_2d,40328960,20070400,9.680279e+06,1.818942e-03,"1x7x7x320,1280x1x1x320,1280",1x7x7x1280,Padding:valid stride:1x1 activation:relu6
66,66,mean,66: mean,0,0,1.732734e+06,1.093961e-05,"1x7x7x1280,2",1x1280,Type=reduceroptions
67,67,fully_connected,67: fully_connected,7680,3840,1.286945e+04,7.112388e-08,"1x1280,3x1280",1x3,Activation:none


In [26]:
layers_df_path = pathlib.Path.joinpath(models_dir, "layer_df_mltk.pkl")
layers_df.to_pickle(layers_df_path)

In [27]:
mltk_model_stats

{'MACs': '299.498 M',
 'FLOPs': '611.424 M',
 'total_params': 2261827,
 'trainable_params': 2227715,
 'non_trainable_params': 34112,
 'model_size_kb': 9299.8984375,
 'tflite_model_size_kb': 8619.51171875,
 'tflite_model_INT8_size_kb': 2602.7890625,
 'arhcitecture': 'mobilenetv2_1_224_c3_o3_keras',
 'alpha': 1.0,
 'img_res': 224,
 'classes': 3,
 'channels': 3,
 'variation': 'keras',
 'opt_RAM_mltk': 9871728,
 'flash_model_size_b_mltk': 8826380,
 'RAM_runtime_memory_size_b_mltk': 9867620,
 'OPS_mltk': 630010831,
 'macs_mltk': 299498112,
 'n_unsupported_layers_mltk': 0,
 'energy_mltk': 0.16587933429054402}

In [28]:
# %%capture optimal_runtime_INT
profiling_results_INT8 = profile_model(str(models_tflite_opt_path), accelerator=None, build=False)

Profiling model in simulator ...
Using Tensorflow-Lite Micro version: b13b48c (2022-06-08)
Searching for optimal runtime memory size ...
Determined optimal runtime memory size to be 2583008


In [29]:
mltk_model_stats["opt_RAM_INT8_mltk"] = int(input("Please copy the optimal runtime memory size from above: \n"))

In [30]:
# from contextlib import redirect_stdout

# with open("runtime.txt", "w", encoding='utf-8') as f:
#     with redirect_stdout(f):
#         print(profile_model(str(models_tflite_opt_path), accelerator=None, build=False))

In [31]:
profiling_results_INT8.get_summary()

OrderedDict([('name', 'mobilenetv2_1_224_c3_o3_keras_INT8'),
             ('accelerator', 'None'),
             ('input_shape', '1x224x224x3'),
             ('input_dtype', 'uint8'),
             ('output_shape', '1x3'),
             ('output_dtype', 'uint8'),
             ('tflite_size', 2665256),
             ('runtime_memory_size', 2578268),
             ('ops', 630612955),
             ('macs', 299498112),
             ('n_layers', 71),
             ('n_unsupported_layers', 0),
             ('cpu_cycles', 841613341.2978516),
             ('cpu_utilization', 0.0),
             ('cpu_clock_rate', 78000000),
             ('energy', 0.16612927192850435),
             ('j_per_op', 2.6344094362683106e-10),
             ('j_per_mac', 5.546922176541278e-10)])

In [32]:
model_profile_INT8 = profiling_results_INT8.to_dict()


In [33]:
mltk_model_stats["flash_model_size_b__INT8_mltk"] = model_profile_INT8["summary"]["tflite_size"]
mltk_model_stats["RAM_runtime_memory_size_b_INT8_mltk"] = model_profile_INT8["summary"]["runtime_memory_size"]
mltk_model_stats["OPS_INT8_mltk"] = model_profile_INT8["summary"]["ops"]
mltk_model_stats["macs_INT8_mltk"] = model_profile_INT8["summary"]["macs"]
mltk_model_stats["n_unsupported_layers_INT8_mltk"] = model_profile_INT8["summary"]["n_unsupported_layers"]
mltk_model_stats["energy_INT8_mltk"] = model_profile_INT8["summary"]["energy"]

In [34]:
mltk_model_stats

{'MACs': '299.498 M',
 'FLOPs': '611.424 M',
 'total_params': 2261827,
 'trainable_params': 2227715,
 'non_trainable_params': 34112,
 'model_size_kb': 9299.8984375,
 'tflite_model_size_kb': 8619.51171875,
 'tflite_model_INT8_size_kb': 2602.7890625,
 'arhcitecture': 'mobilenetv2_1_224_c3_o3_keras',
 'alpha': 1.0,
 'img_res': 224,
 'classes': 3,
 'channels': 3,
 'variation': 'keras',
 'opt_RAM_mltk': 9871728,
 'flash_model_size_b_mltk': 8826380,
 'RAM_runtime_memory_size_b_mltk': 9867620,
 'OPS_mltk': 630010831,
 'macs_mltk': 299498112,
 'n_unsupported_layers_mltk': 0,
 'energy_mltk': 0.16587933429054402,
 'opt_RAM_INT8_mltk': 2583008,
 'flash_model_size_b__INT8_mltk': 2665256,
 'RAM_runtime_memory_size_b_INT8_mltk': 2578268,
 'OPS_INT8_mltk': 630612955,
 'macs_INT8_mltk': 299498112,
 'n_unsupported_layers_INT8_mltk': 0,
 'energy_INT8_mltk': 0.16612927192850435}

In [35]:
layers_INT8_df = pd.DataFrame.from_dict(model_profile_INT8["layers"])
layers_INT8_df["name"] = layers_INT8_df["index"].map(str) + ": " + layers_INT8_df["opcode"]
layers_INT8_df.insert(2,"name" , layers_INT8_df.pop("name"))
layers_INT8_df

,index,opcode,name,ops,macs,cpu_cycles,energy,input_shape,output_shape,options
0,0,quantize,0: quantize,602112,0,1.283454e+06,2.495152e-04,1x224x224x3,1x224x224x3,Type=none
1,1,conv_2d,1: conv_2d,22880256,10838016,1.352801e+08,2.802536e-02,"1x224x224x3,32x3x3x3,32",1x112x112x32,Padding:same stride:2x2 activation:relu6
2,2,depthwise_conv_2d,2: depthwise_conv_2d,8429568,3612672,1.429125e+07,2.795809e-03,"1x112x112x32,1x3x3x32,32",1x112x112x32,Multiplier:1 padding:same stride:1x1 activatio...
3,3,conv_2d,3: conv_2d,13045760,6422528,1.957062e+07,3.881317e-03,"1x112x112x32,16x1x1x32,16",1x112x112x16,Padding:same stride:1x1 activation:none
4,4,conv_2d,4: conv_2d,42147840,19267584,4.246512e+07,8.768505e-03,"1x112x112x16,96x1x1x16,96",1x112x112x96,Padding:same stride:1x1 activation:relu6
...,...,...,...,...,...,...,...,...,...,...
66,66,conv_2d,66: conv_2d,40328960,20070400,9.680279e+06,1.818942e-03,"1x7x7x320,1280x1x1x320,1280",1x7x7x1280,Padding:valid stride:1x1 activation:relu6
67,67,mean,67: mean,0,0,1.732734e+06,1.093961e-05,"1x7x7x1280,2",1x1280,Type=reduceroptions
68,68,fully_connected,68: fully_connected,7680,3840,1.286945e+04,7.112388e-08,"1x1280,3x1280",1x3,Activation:none
69,69,softmax,69: softmax,15,0,3.838418e+03,1.652612e-08,1x3,1x3,Type=softmaxoptions


In [36]:
layers_df_INT8_path = pathlib.Path.joinpath(models_dir, "layer_df_INT8_mltk.pkl")
layers_INT8_df.to_pickle(layers_df_INT8_path)

In [37]:
#layers_INT8_df.info()

# Getting peak memory infos

In [38]:
models_peak_memory_path = models_dir.joinpath(model_name, f"{model_name}_peak_memory")

In [39]:
! python i:\tinyml\tflite-tools\tflite_tools.py -i $models_tflite_opt_path --csv $models_peak_memory_path
! explorer $models_peak_memory_path

Writing model analysis to i:\tinyml\tiny_cnn\models\mobilenetv2_1_224_c3_o3_keras\mobilenetv2_1_224_c3_o3_keras_peak_memory in CSV format


In [40]:
model_peak_memory_schedule = models_dir.joinpath(model_name, f"{model_name}_peak_memory", "execution_schedule_info.csv")
peak_memory_tensor_details = models_dir.joinpath(model_name, f"{model_name}_peak_memory", "tensor_details.csv")

In [41]:
peak_memory_df = get_peak_memory_df(model_peak_memory_schedule)
peak_memory_df.head(10)

Reading in i:\tinyml\tiny_cnn\models\mobilenetv2_1_224_c3_o3_keras\mobilenetv2_1_224_c3_o3_keras_peak_memory\execution_schedule_info.csv
Cleaning up the dataframe.


,name,tensor_IDs,RAM_b,operator,index
0,tfl.quantize,"(0, 108)",301056,tfl.quantize,0
1,Conv1,"(108, 109)",551936,mobilenetv2_1.00_224/Conv1_relu/Relu6;mobilene...,1
2,block_5_project_BN,"(109, 110)",802816,mobilenetv2_1.00_224/expanded_conv_depthwise_r...,2
3,expanded_conv_project,"(110, 111)",602112,mobilenetv2_1.00_224/expanded_conv_project_BN/...,3
4,block_1_expand,"(111, 112)",1404928,mobilenetv2_1.00_224/block_1_expand_relu/Relu6...,4
5,block_1_pad,"(112, 113)",2430048,mobilenetv2_1.00_224/block_1_pad/Pad,5
6,block_12_project_BN,"(113, 114)",1526880,mobilenetv2_1.00_224/block_1_depthwise_relu/Re...,6
7,block_1_project,"(114, 115)",376320,mobilenetv2_1.00_224/block_1_project_BN/FusedB...,7
8,block_2_expand,"(115, 116)",526848,mobilenetv2_1.00_224/block_2_expand_relu/Relu6...,8
9,block_3_depthwise_BN,"(115, 116, 117)",978432,mobilenetv2_1.00_224/block_2_depthwise_relu/Re...,9


In [42]:
peak_memory = peak_memory_df["RAM_b"].max()
peak_memory

2430048

In [43]:
mltk_model_stats["peak_memory_b"] = peak_memory

In [44]:
peak_memory_df["RAM_b"].argmax()

5

In [45]:
tensor_df = get_tensor_details_df(peak_memory_tensor_details)
tensor_df

Reading in i:\tinyml\tiny_cnn\models\mobilenetv2_1_224_c3_o3_keras\mobilenetv2_1_224_c3_o3_keras_peak_memory\tensor_details.csv
Cleaning up the dataframe.


,id,name,shape,size_b,name_long
0,0,serving_default_input_1:0,"(1, 224, 224, 3)",150528,serving_default_input_1:0
1,108,tfl.quantize,"(1, 224, 224, 3)",150528,tfl.quantize
2,109,mobilenetv2_1.00_224/Conv1/Conv2D,"(1, 112, 112, 32)",401408,mobilenetv2_1.00_224/Conv1_relu/Relu6;mobilene...
3,110,mobilenetv2_1.00_224/block_5_project_BN/FusedB...,"(1, 112, 112, 32)",401408,mobilenetv2_1.00_224/expanded_conv_depthwise_r...
4,111,mobilenetv2_1.00_224/expanded_conv_project/Conv2D,"(1, 112, 112, 16)",200704,mobilenetv2_1.00_224/expanded_conv_project_BN/...
...,...,...,...,...,...
67,174,mobilenetv2_1.00_224/Conv_1/Conv2D,"(1, 7, 7, 1280)",62720,mobilenetv2_1.00_224/out_relu/Relu6;mobilenetv...
68,175,mobilenetv2_1.00_224/global_average_pooling2d/...,"(1, 1280)",1280,global_average_pooling2d
69,176,mobilenetv2_1.00_224/predictions/BiasAdd,"(1, 3)",3,mobilenetv2_1.00_224/predictions/MatMul;mobile...
70,177,StatefulPartitionedCall:01,"(1, 3)",3,StatefulPartitionedCall:01


# Logging to Weights & Biases

In [46]:
api = wandb.Api()

# Generate run ids
run_id = wandb.util.generate_id()

wandb.init(
    project=PROJECT,
    entity="susbrock",
    id=run_id,
    resume=True)
config = wandb.config
config.id = run_id
config.update(mltk_model_stats)


layers_df_wandb = wandb.Table(dataframe=layers_df)
wandb.log({"tflite_layers_mltk" : layers_df_wandb})
layers_INT8_df_wandb = wandb.Table(dataframe=layers_INT8_df)
wandb.log({"tflite_INt8_layers_mltk" : layers_INT8_df_wandb})

peak_memory_df_wandb = wandb.Table(dataframe=peak_memory_df)
wandb.log({"peak_memory" : peak_memory_df_wandb})

tensor_df_wandb = wandb.Table(dataframe=tensor_df)
wandb.log({"tensor_info" : tensor_df_wandb})



wandb.log({"tflite_INt8_layers_plt" : wandb.plot.bar(layers_INT8_df_wandb, "index", "macs", title=model_name)})

wandb.finish()



ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: susbrock. Use `wandb login --relogin` to force relogin
